In [2]:
import turicreate as tc
import pandas as pd
%load_ext sql
from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import numpy as np

In [3]:
books_vidacha = pd.read_csv('data/books_vydacha.csv')

In [4]:
books_exemplyary = pd.read_csv('data/books_exemplyary.csv')

In [5]:
books_catalog = pd.read_csv('data/books_catalog.csv')

In [26]:
kdf_pupils = pd.read_csv('data/kdf_pupils.csv')

In [27]:
kdf_mergerelation = pd.read_csv('data/kdf_mergerelation.csv')

In [3]:
kdf_organization = pd.read_csv('data/kdf_organization.csv')

In [30]:
kdf_services = pd.read_csv('data/kdf_services.csv')

In [7]:
events = pd.read_csv('data/meropriyatia.csv', sep=';')

In [32]:
metro_distance = pd.read_csv('data/metro_distance.csv')

# Turicreate baseline книги

In [6]:
books_vidacha['id_katalognoi'] = books_vidacha['inventarnyi_nomer'].map(books_exemplyary.set_index('inventarnyi_nomer')['id_katalognoi'])
books_vidacha['doc_id'] = books_vidacha['id_katalognoi']

In [7]:
books_total = books_vidacha.merge(books_catalog,on='doc_id')

In [8]:
books_total = books_total.dropna(subset=['id_chitatelya'])

In [9]:
books_total = books_total.drop_duplicates(subset=['id_chitatelya','inventarnyi_nomer','id_katalognoi'])

In [10]:
books_total['id_chitatelya'] = books_total['id_chitatelya'].astype(str)
books_total['doc_id'] = books_total['doc_id'].astype(str)

## Cleaning

In [11]:
books_catalog_clean = books_catalog.dropna()

In [12]:
books_catalog_clean['author'] = books_catalog_clean['author'].astype(str)
books_catalog_clean['doc_id'] = books_catalog_clean['doc_id'].astype(str)
books_catalog_clean['tags'] = books_catalog_clean['tags'].astype(str)

In [13]:
tc_books = tc.SFrame(books_catalog_clean)

In [19]:
columns = ['id_chitatelya',
'doc_id']
item_columns = ['doc_id',
 'author']

In [20]:
tc_books_total = tc.SFrame(books_total[['id_chitatelya','doc_id']])

In [21]:
model = tc.recommender.ranking_factorization_recommender.create(tc_books_total[columns], 
                                                        user_id="id_chitatelya", random_seed=None,
                                                        item_id="doc_id",
                                                        max_iterations=120,solver='adagrad',
                                                        item_data=tc_books[item_columns],
                                                                verbose=True)

Preparing data set.

Data has 2990 observations with 2138 users and 193552 items.

Data prepared in: 0.350831s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.240523

Final training Predictive Error: 0.0970471

In [22]:
recommended = model.recommend(tc_books_total["id_chitatelya"].unique(), k=3, exclude_known=True)

recommendations finished on 1000/2138 queries. users per second: 78.2701

recommendations finished on 2000/2138 queries. users per second: 80.4168

In [23]:
recommended

id_chitatelya,doc_id,score,rank
557016,1454366,1.0,1
557016,1451659,1.0,2
557016,1465737,1.0,3
421643,1454366,1.0,1
421643,1465737,1.0,2
421643,1473610,1.0,3
312796,1454438,1.0,1
312796,1451659,1.0,2
312796,1465887,1.0,3
497557,1451659,1.0,1


In [24]:
recoms = recommended.to_dataframe()

In [25]:
books_catalog_check = books_catalog[['doc_id','author','tags']]

# Turicreate baseline кружки

In [31]:
kdf_mergerelation['merge_column'] = kdf_mergerelation['id_yslugi'].map(kdf_services.set_index('id_yslugi')['merge_column'])

## Get often metro station

In [32]:
kdf_mergerelation['metro'] = kdf_mergerelation['id_org'].map(kdf_organization.drop_duplicates(subset=['id_org']).set_index('id_org')['metro_new'])

In [33]:
fav_metro = kdf_mergerelation.dropna().drop_duplicates(subset=['id_puiple','id_org']).groupby('id_puiple')['metro'].agg(lambda x: pd.Series.mode(x).iat[0]).to_frame()

In [88]:
fav_metro['id_puiple'] = fav_metro.index

In [89]:
fav_metro['id_puiple'] = fav_metro['id_puiple'].astype(str)
fav_metro['metro'] = fav_metro['metro'].astype(str)

In [90]:
tc_fav_metro = tc.SFrame(fav_metro[['id_puiple','metro']])

In [38]:
columns_kdf = ['id_puiple',
'merge_column']

In [39]:
kdf_mergerelation['id_puiple'] = kdf_mergerelation['id_puiple'].astype(str)
kdf_mergerelation['merge_column'] = kdf_mergerelation['merge_column'].astype(str)

In [40]:
tc_kdf_total = tc.SFrame(kdf_mergerelation[['id_puiple','merge_column']])

In [41]:
model_kdf = tc.recommender.ranking_factorization_recommender.create(tc_kdf_total[columns_kdf], 
                                                        user_id="id_puiple", random_seed=None,
                                                        item_id="merge_column",
                                                        max_iterations=120,solver='adagrad',
                                                                verbose=True)

Preparing data set.

Data has 724813 observations with 289170 users and 16281 items.

Data prepared in: 0.860742s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.00997819

Final training Predictive Error: 0.00148233

In [42]:
recommended_kdf = model_kdf.recommend(tc_kdf_total["id_puiple"].unique(), k=10, exclude_known=True)

recommendations finished on 1000/289170 queries. users per second: 6753.15

recommendations finished on 2000/289170 queries. users per second: 6655.57

recommendations finished on 3000/289170 queries. users per second: 6586.17

recommendations finished on 4000/289170 queries. users per second: 6599.22

recommendations finished on 5000/289170 queries. users per second: 6582.59

...

recommendations finished on 285000/289170 queries. users per second: 6997.21

recommendations finished on 286000/289170 queries. users per second: 6998.34

recommendations finished on 287000/289170 queries. users per second: 6999.52

recommendations finished on 288000/289170 queries. users per second: 7000.76

recommendations finished on 289000/289170 queries. users per second: 7001.14

In [44]:
recommended_kdf_df = recommended_kdf.to_dataframe()

## Sort - metro distance

In [91]:
recommended_kdf_df['fav_metro'] = recommended_kdf_df['id_puiple'].map(fav_metro['metro'])

In [93]:
recommended_kdf_df['metro']= recommended_kdf_df['merge_column'].map(kdf_mergerelation.drop_duplicates(subset=['merge_column']).set_index('merge_column')['metro'])

In [94]:
metro_distance['concat_metro'] = metro_distance['metro_1'] + '_' + metro_distance['metro_2']

In [95]:
recommended_kdf_df['concat_metro'] = recommended_kdf_df['fav_metro'] + '_' + recommended_kdf_df['metro'] 

In [96]:
recommended_kdf_df['euklid'] = recommended_kdf_df['concat_metro'].map(metro_distance.drop_duplicates(subset=['concat_metro']).set_index('concat_metro')['eculid'])

In [99]:
recommended_kdf_df[recommended_kdf_df['fav_metro']==recommended_kdf_df['metro']]['euklid'].unique()

array([ 0., nan])

In [145]:
recommended_kdf_df_sort = recommended_kdf_df_sort.sort_values(['id_puiple','euklid'], ascending=True)

In [147]:
recommended_kdf_df_sort[recommended_kdf_df_sort['id_puiple']=='1287665']

,id_puiple,merge_column,score,rank,fav_metro,metro,concat_metro,euklid
551533,1287665,162_Ударные инструменты,0.352001,4,Сокольники,Сокольники,Сокольники_Сокольники,0.000000
551535,1287665,162_Академический вокал,0.245126,6,Сокольники,Сокольники,Сокольники_Сокольники,0.000000
551532,1287665,48_Блок-флейта,0.360070,3,Сокольники,Бабушкинская,Сокольники_Бабушкинская,0.082832
551534,1287665,10872_Московский городской творческий коллекти...,0.314051,5,Сокольники,Текстильщики,Сокольники_Текстильщики,0.095596
551539,1287665,30_Архитектура,0.181357,10,Сокольники,Баррикадная,Сокольники_Баррикадная,0.102684
551537,1287665,10856_Топотушки – подготовительная группа в де...,0.199580,8,Сокольники,Алтуфьево,Сокольники_Алтуфьево,0.144129
551531,1287665,10907_ Студия любителей английского языка Линг...,0.482623,2,Сокольники,Алма-Атинская,Сокольники_Алма-Атинская,0.177848
551530,1287665,"10858_Вокальная студия Лирика (хор), платная о...",0.511436,1,Сокольники,NaN,NaN,NaN
551536,1287665,21_Форт8,0.239570,7,Сокольники,NaN,NaN,NaN
551538,1287665,10867_Студия изобразительного искусства «Вдох...,0.188093,9,Сокольники,NaN,NaN,NaN


# LDA по тегам книг

In [125]:
books_catalog_check['tags'].unique()

array(['История в целом. Всемирная история', 'Вокальные произведения',
       'Художественная литература', ...,
       'СМИ. Журналистика в целом ; Художественно-документальная проза. Мемуары',
       'История киноискусства ; Драматический театр',
       'История и критика литературы России ; История и критика мировой литературы ; История и критика литературы зарубежных стран'],
      dtype=object)

In [126]:
books_catalog_check_new = books_catalog_check.dropna(subset=['tags'])

In [128]:
stop = stopwords.words('russian')
books_catalog_check_new['new_tags'] = books_catalog_check_new['tags'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 

In [129]:
stop2 = ['литература','произведения']
books_catalog_check_new['new_tags'] = books_catalog_check_new['new_tags'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop2)])) 

In [130]:
books_catalog_check_new['new_tags'] = books_catalog_check_new['new_tags'].str.replace(r'(\<(/?[^>]+)>)',' ').replace(r'[\W_]+',' ')

In [131]:
count_vectorizer = CountVectorizer(max_features=10000)
books_tags = books_catalog_check_new['new_tags'].values
document_term_matrix = count_vectorizer.fit_transform(books_tags)


In [132]:
n_topics = 14
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)


In [134]:
# Define helper functions
def get_keys(topic_matrix):
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):

    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [137]:
lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

In [138]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):

    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word)
        top_words.append(" ".join(topic_words))         
    return top_words


In [139]:
top_n_words_lsa = get_top_n_words(10, lsa_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

Topic 1:  художественная детей юношества российская проза зарубежная фантастика детектив зарубежный поэзия
Topic 2:  история россии критика литературы теория целом всемирная музыки стран зарубежных
Topic 3:  детей юношества художественная зоология транспорт зарубежная российская проза культура рукоделие
Topic 4:  право семейное гражданское финансовое торговое интеллектуальное юридические науки исполнительное судопроизводство
Topic 5:  целом геология журналистика сми религия религиоведение терапия анастезиология реанимация интенсивная
Topic 6:  экономика бизнес экономические стран регионов предприятия предпринимательство политика мировая мировые
Topic 7:  живопись науки психология издания искусство универсального содержания социальная земле естественные
Topic 8:  охрана искусства памятников собрания художественные музеи коллекции графика скульптура культуры
Topic 9:  музыка языки мира яыкознание частное театр архитектура градостроительство драматический программирования
Topic 10:  общие

In [153]:
top_n_words_lsa_last = pd.DataFrame(top_n_words_lsa)

In [154]:
top_n_words_lsa_last['topic'] = top_n_words_lsa_last.index

In [155]:
top_n_words_lsa_last

,0,topic
0,художественная детей юношества российская проз...,0
1,история россии критика литературы теория целом...,1
2,детей юношества художественная зоология трансп...,2
3,право семейное гражданское финансовое торговое...,3
4,целом геология журналистика сми религия религи...,4
5,экономика бизнес экономические стран регионов ...,5
6,живопись науки психология издания искусство ун...,6
7,охрана искусства памятников собрания художеств...,7
8,музыка языки мира яыкознание частное театр арх...,8
9,общие вопросы литературоведения других музыкал...,9


In [158]:
books_catalog_check_new['num_topic'] = lsa_keys
books_catalog_check_new['keywords'] = books_catalog_check_new['num_topic'].map(top_n_words_lsa_last.set_index('topic')[0])

## LSA - связываем топики по книгам с мероприятиями

In [161]:
events['new_type_meropriyatia'] = events['type_meropriyatia'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 

## Word2Vec для книги-мероприятия

In [162]:
import gensim.downloader as api

pre_train_model = api.load('word2vec-ruscorpora-300')

In [164]:
len(pre_train_model.wv.vocab)

184973

In [165]:
items = pre_train_model.wv.vocab.items()
for key, value in list(items):
    index = str.find(key, "_")
    key_new = key[:index]
    pre_train_model.wv.vocab[key_new] = value
    del pre_train_model.wv.vocab[key]

In [173]:
pre_train_model.wv.most_similar(positive=["культура"])

[('культурный_ADJ', 0.6627464294433594),
 ('сверхкультурный_ADJ', 0.5889217853546143),
 ('цивилизация_NOUN', 0.5629868507385254),
 ('антикультура_NOUN', 0.5456435084342957),
 ('аденокарцинома::надпочечник_NOUN', 0.540786623954773),
 ('искусство_NOUN', 0.5383590459823608),
 ('тритикале_NOUN', 0.5383437871932983),
 ('mrc_NOUN', 0.535457968711853),
 ('диплоидный::фибробласт_NOUN', 0.5242698192596436),
 ('этнокультура_NOUN', 0.5211303234100342)]

In [204]:
keys = pre_train_model.wv.vocab.keys()
for j in events['new_type_meropriyatia'].str.split(',').str[0].str.split(' ').str[0]:
    if(j in keys):
        rec_events = pre_train_model.wv.most_similar(positive=[о])

In [166]:
pre_train_model.wv.vocab

nsim.models.keyedvectors.Vocab at 0x1a65154bd0>,
 'кабинет': <gensim.models.keyedvectors.Vocab at 0x1a65154c10>,
 'возможный': <gensim.models.keyedvectors.Vocab at 0x1a65154c50>,
 'немецкий': <gensim.models.keyedvectors.Vocab at 0x1a65154c90>,
 'фронт': <gensim.models.keyedvectors.Vocab at 0x1a65154cd0>,
 'удар': <gensim.models.keyedvectors.Vocab at 0x1a65154d10>,
 'политика': <gensim.models.keyedvectors.Vocab at 0x1a65154d50>,
 'переставать': <gensim.models.keyedvectors.Vocab at 0x1a65154d90>,
 'внешний': <gensim.models.keyedvectors.Vocab at 0x1a65154dd0>,
 'ситуация': <gensim.models.keyedvectors.Vocab at 0x1a65154e10>,
 'номер': <gensim.models.keyedvectors.Vocab at 0x1a65154e50>,
 'выбирать': <gensim.models.keyedvectors.Vocab at 0x1a65154e90>,
 'либо': <gensim.models.keyedvectors.Vocab at 0x1a65154ed0>,
 'тема': <gensim.models.keyedvectors.Vocab at 0x1a65154f10>,
 'граф': <gensim.models.keyedvectors.Vocab at 0x1a65154f50>,
 'легкий': <gensim.models.keyedvectors.Vocab at 0x1a65154f90>